In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from localisation.locfunction import *

import pandas as pd
import numpy as np

n=100
noise=3000
flux=2780
NSIDE = 128
panels=5
database=[]
chi2_result=[]
vector_result=[]
true_GRB_ra, true_GRB_dec = [],[]

GRB_ra_array,GRB_dec_array = isotropicpoints(n)

for GRB_ra,GRB_dec in zip(GRB_ra_array,GRB_dec_array):
    counts = observedcounts(GRB_ra,GRB_dec,flux,noise)

    vec_result = vectorlocalisation(counts,noise)
    if vec_result is None:
        continue

    chira,chidec,chiflux = chi2localisation(counts,NSIDE,panels,noise)
    chi2_result.append((chira,chidec,chiflux))

    vecra,vecdec,vecflux = vectorlocalisation(counts,noise)
    vector_result.append((vecra,vecdec,vecflux))

    database.append({'Counts': counts,'GRB_ra': GRB_ra,'GRB_dec': GRB_dec})

    true_GRB_ra.append(GRB_ra)
    true_GRB_dec.append(GRB_dec)

#df=pd.DataFrame(database)
#df.to_csv('100uniformcounts.csv')

chi2_ra,chi2_dec,chi2_flux = np.array(chi2_result).T
vec_ra,vec_dec,vec_flux = np.array(vector_result).T

#errors
chi2ra_error = chi2_ra- np.array(true_GRB_ra)
chi2dec_error = chi2_dec - np.array(true_GRB_dec)
chi2flux_error = chi2_flux - flux

vecra_error = vec_ra-np.array(true_GRB_ra)
vecdec_error = vec_dec - np.array(true_GRB_dec)
vecflux_error = vec_flux - flux 

bins=round(np.sqrt(vecflux_error.shape[0]))

plotdistribution(chi2ra_error,vecra_error,'Chi2','Vector',0,'RA error','difference',bins)
plotdistribution(chi2dec_error,vecdec_error,'Chi2','Vector',0,'Dec error','difference',bins)
plotdistribution(chi2flux_error,vecflux_error,'Chi2','Vector',0,'Flux error','difference',bins)


In [ ]:
GRB_ra, GRB_dec = isotropicpoints(5000)
import matplotlib.pyplot as plt

theta = np.radians(90 - GRB_dec)
#phi = np.radians(GRB_ra)
plt.hist(np.cos(theta), bins=50)
plt.xlabel("cos(theta)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

from localisation.locfunction import *

import numpy as np

n=100
noise=3000
flux=2780
NSIDE = 128
panels=5
injection_dict = {}
error_dict = {}  
results_dict = {}  

GRB_ra_array,GRB_dec_array = isotropicpoints(n)

for GRB_ra,GRB_dec in zip(GRB_ra_array,GRB_dec_array):
    injections=[]
    errors_list=[]
    results_list=[]
    for _ in range(1000):
        counts = observedcounts(GRB_ra,GRB_dec,flux,noise)
        injections.append(counts)

        vec_result = vectorlocalisation(counts,noise)
        if vec_result is None:
            continue

        chira,chidec,chiflux = chi2localisation(counts,NSIDE,panels,noise)

        vecra,vecdec,vecflux = vectorlocalisation(counts,noise)

        error_row = [
            chira -GRB_ra,
            chidec - GRB_dec,
            chiflux - flux,
            vecra - GRB_ra,
            vecdec- GRB_dec,
            vecflux-flux
        ]
        errors_list.append(error_row)

        result_row = [
            GRB_ra, GRB_dec,
            chira, chidec, chiflux,
            vecra, vecdec, vecflux
        ]
        results_list.append(result_row)
    
    key = f"{GRB_ra:.5f}_{GRB_dec:.5f}"
    error_dict[key] = np.array(errors_list)
    results_dict[key] = np.array(results_list)
    injection_dict[key] = np.array(injections)


np.savez("injections.npz", **injection_dict)
np.savez("sourcewise_errors.npz", **error_dict)
np.savez("sourcewise_results.npz", **results_dict)

In [7]:
import numpy as np
data= np.load("injections.npz")
print(data.files)

['194.94030_48.27685', '116.79416_23.18317', '92.77524_6.55298', '114.46516_12.83197', '305.18148_21.29417', '50.70658_49.42316', '57.80473_43.02992', '180.67714_3.61692', '81.60303_28.12787', '296.50080_33.10995', '195.59201_0.66326', '124.18344_30.13552', '31.32001_64.91248', '334.77948_3.50277', '124.26853_40.78595', '242.85823_64.75020', '316.77185_32.98300', '24.17327_12.02230', '170.87539_78.56646', '30.52676_17.56550', '56.40705_4.04226', '13.09502_17.67366', '292.59582_67.66765', '83.06670_58.55480', '172.14608_62.72727', '282.68721_9.99373', '283.96807_7.74137', '185.41330_32.71834', '214.10135_21.26407', '272.80871_29.14582', '318.95094_27.64618', '292.94336_45.48607', '53.56264_19.89577', '229.20332_62.18740', '22.11652_57.92252', '148.48034_64.17477', '272.42075_64.82952', '168.19224_17.18112', '275.75764_28.13942', '155.79097_16.01801', '136.95712_51.11070', '198.21199_28.49732', '42.79564_23.65383', '304.79881_60.02083', '83.60689_79.25148', '248.36998_40.94730', '334.075

In [ ]:

print(err.files)

['194.94030_48.27685', '116.79416_23.18317', '92.77524_6.55298', '114.46516_12.83197', '305.18148_21.29417', '50.70658_49.42316', '57.80473_43.02992', '180.67714_3.61692', '81.60303_28.12787', '296.50080_33.10995', '195.59201_0.66326', '124.18344_30.13552', '31.32001_64.91248', '334.77948_3.50277', '124.26853_40.78595', '242.85823_64.75020', '316.77185_32.98300', '24.17327_12.02230', '170.87539_78.56646', '30.52676_17.56550', '56.40705_4.04226', '13.09502_17.67366', '292.59582_67.66765', '83.06670_58.55480', '172.14608_62.72727', '282.68721_9.99373', '283.96807_7.74137', '185.41330_32.71834', '214.10135_21.26407', '272.80871_29.14582', '318.95094_27.64618', '292.94336_45.48607', '53.56264_19.89577', '229.20332_62.18740', '22.11652_57.92252', '148.48034_64.17477', '272.42075_64.82952', '168.19224_17.18112', '275.75764_28.13942', '155.79097_16.01801', '136.95712_51.11070', '198.21199_28.49732', '42.79564_23.65383', '304.79881_60.02083', '83.60689_79.25148', '248.36998_40.94730', '334.075

In [34]:
err = np.load("sourcewise_errors.npz")
keys = list(err.files)

source=keys[0]
df = err[source]

'''
ra_err_chi2 = df[:,0]
dec_err_chi2 = df[:,1]
flux_err_chi2 = df[:,2]
ra_err_vec = df[:,3]
dec_err_vec = df[:,4]
flux_err_vec = df[:,5]

#plotdistribution(ra_err_chi2,ra_err_vec,'RA Error Chi2','RA Error Vec',0,'RA Error','Error')
'''

"\nra_err_chi2 = df[:,0]\ndec_err_chi2 = df[:,1]\nflux_err_chi2 = df[:,2]\nra_err_vec = df[:,3]\ndec_err_vec = df[:,4]\nflux_err_vec = df[:,5]\n\n#plotdistribution(ra_err_chi2,ra_err_vec,'RA Error Chi2','RA Error Vec',0,'RA Error','Error')\n"

In [35]:
df

array([[-12.85697098,   1.42554391, 108.26042872,   7.37642092,
         -6.16657995,  86.26292409]])

In [25]:
print(type(df))
print(df.shape)
print(df)


<class 'numpy.ndarray'>
(1, 6)
[[-12.85697098   1.42554391 108.26042872   7.37642092  -6.16657995
   86.26292409]]
